In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import os
import shutil

if os.path.exists('./file/4.description'):
    shutil.rmtree('./file/4.description')

os.makedirs('./file/4.description')

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import re

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/2.item'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

for file in list_file:
    
    # = = = = = = = = = = = = = = =

    output_correct, output_error = pd.DataFrame(), pd.DataFrame()

    # = = = = = = = = = = = = = = =

    input_ = pd.read_excel(f'./file/2.item/{file}',
                           header=0,
                           dtype=str).fillna('')

    input_['No'] = input_['No'].astype(int)

    print(f'总数量：{len(input_)}')
    print()

    # = = = = = = = = = = = = = = =


    work = Queue()
    for a in range(len(input_)):
        work.put_nowait(a)

    # = = = = = = = = = = = = = = =

    def crawler():
        global output_correct, output_error

        while not work.empty():
            a = work.get_nowait()

            # = = = = = = = = = = = = = = =

            crawler_status = 'error'

            # = = = = = = = = = = = = = = =

            try:
                b = 0
                while True:
                    b += 1

                    try:
                        resp = requests.get(input_.loc[a, 'Description Url'],
                                            headers=get_header(ua=False),
                                            proxies=get_proxy(),
                                            timeout=(10, 10))
            
                        if resp.status_code == 200:
                            break
                    except KeyboardInterrupt:
                        break
                    except:
                        continue
            
                # = = = = = = = = = = = = = = =
            
                soup = BeautifulSoup(resp.text, 'lxml')
                html = etree.HTML(str(soup))

                # = = = = = = = = = = = = = = =
            
                list_text = html.xpath('//div[@class="x-item-description-child"]/center/table/tbody/tr[2]/td/table[2]/tbody/tr[2]/td[@class="text"]/div/descendant-or-self::strong')
                list_text = [etree.tostring(text).decode('utf-8') for text in list_text]
                list_text = [re.sub(r'(<br/>)+', '<br/>', text) for text in list_text]
                list_text = [re.sub(r'<.*?>', ' ', text.replace('<br/>', '\n')) for text in list_text]
                list_text = [re.sub(r' +', ' ', text).strip() for text in list_text]

                # = = = = = = = = = = = = = = =

                df_temp = pd.DataFrame([{'No': input_.loc[a, 'No'],
                                         'Item Number': input_.loc[a, 'Item Number'],
                                         'Description Url': input_.loc[a, 'Description Url']}])
                
                for i, text in zip(range(len(list_text)), list_text):
                    df_temp.loc[0, f'Text {i+1}'] = text
                
                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =

                crawler_status = 'ok'
            except:
                output_error = pd.concat([output_error, pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)], ignore_index=True).fillna('')


            # = = = = = = = = = = = = = = =

            if list_file.index(file) % 3 == 0 and list_file.index(file) != 0 and work.qsize() == 0 or work.qsize() % 1_000 == 0 and work.qsize() != 0:
                clear_output()
    
            print(f'''[状态：{crawler_status}，尝试次数：{b}] - {(list_file.index(file) + 1) / len(list_file) * 100:.2f}% - {file.removesuffix('.xlsx')} > {input_.loc[a, 'No']}.{input_.loc[a, 'Item Number']}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

    # = = = = = = = = = = = = = = =

    list_task = []
    for _ in range(5):
        task = gevent.spawn(crawler)
        list_task.append(task)
    gevent.joinall(list_task)

    print('输出ing...')
    print()
    if not output_correct.empty:
        output_correct = output_correct.sort_values(by=['No'],
                                                    ascending=[True],
                                                    ignore_index=True)
        output_correct.to_excel(f'''./file/4.description/{file.removesuffix('.xlsx')}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
    if not output_error.empty:
        output_error = output_error.sort_values(by=['No'],
                                                ascending=[True],
                                                ignore_index=True)
        output_error.to_excel(f'''./file/{file.removesuffix('.xlsx')}-description_error.xlsx''', index=False)
        print('爬虫存在error')
        print()

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/4.description'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

from tqdm import tqdm
import re

for file in tqdm(list_file, desc='Progress', ncols=77):
    os.rename(f'./file/4.description/{file}',
              f'''./file/4.description/{re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '.xlsx', file)}''')

print('Done ~')

总数量：418

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 1.235461387733
[剩余数量：413] - [当前时间：21:36:09]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 2.233621258535
[剩余数量：412] - [当前时间：21:36:10]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 3.334922305483
[剩余数量：411] - [当前时间：21:36:10]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 4.255586148611
[剩余数量：410] - [当前时间：21:36:11]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 9.235443864456
[剩余数量：409] - [当前时间：21:36:13]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 6.232315264904
[剩余数量：408] - [当前时间：21:36:14]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 7.235123315262
[剩余数量：407] - [当前时间：21:36:15]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 8.256372876426
[剩余数量：406] - [当前时间：21:36:16]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 11.256291287123
[剩余数量：405] - [当前时间：21:36:18]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 13.235544392154
[剩余数量：404] - [当前时间：21:36:18]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 14.335057592559
[剩余数量：403] - [当前时间：21:36:20]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 12.335372891937
[剩余数量：402] - [当前时间：21

KeyboardInterrupt
2024-08-12T13:36:32Z


KeyboardInterrupt: 